<a href="https://colab.research.google.com/github/BBARRY-Lee/NLP/blob/main/KoBERT%EB%A5%BC_%ED%99%9C%EC%9A%A9%ED%95%9C_%EA%B0%90%EC%A0%95%EB%B6%84%EB%A5%98_%EB%AA%A8%EB%8D%B8_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Environment Setting

In [1]:
import torch
print(torch.cuda.is_available())
device = torch.device("cuda:0")

True


In [2]:
# install libraries: colab environment
!pip install datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.2 MB/s eta 0:00:00


---

# 2. Data preprocessing

## 2.1 Get AI Hub Data

In [37]:
import pandas as pd
df = pd.read_csv("./감정_데이터셋_1.csv", encoding='cp949')

In [38]:
df[:5]

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,5f4141e29dd513131eacee2f,헐! 나 이벤트에 당첨 됐어.,happiness,angry,2,surprise,2,happiness,2,happiness,2,happiness,2,48,female
1,5f4141f59dd513131eacee30,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,happiness,neutral,0,happiness,2,happiness,2,happiness,2,happiness,2,48,female
2,5f4142119dd513131eacee31,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,2,48,female
3,5f4142279dd513131eacee32,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,1,48,female
4,5f3c9ed98a3c1005aa97c4bd,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,neutral,happiness,2,happiness,1,happiness,2,happiness,1,happiness,1,48,female


In [39]:
df.shape

(19374, 15)

### Remove duplicates

In [40]:
s = df.duplicated(['발화문'], keep=False)
df[s]

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,5f4141e29dd513131eacee2f,헐! 나 이벤트에 당첨 됐어.,happiness,angry,2,surprise,2,happiness,2,happiness,2,happiness,2,48,female
27,5f41443c9dd513131eacee41,나 축하해줘. 기분 좋은 일이 생겼어.,happiness,happiness,2,happiness,2,happiness,2,happiness,2,happiness,2,48,female
52,5f4147619dd513131eacee61,드디어 구매한 노트북이 도착했어.,happiness,neutral,0,happiness,2,happiness,1,happiness,1,happiness,2,48,female
62,5f4149139dd513131eacee7a,나 드디어 프로젝트 끝냈어!,happiness,happiness,2,happiness,2,happiness,2,happiness,2,happiness,2,48,female
76,5f414b109dd513131eacee8d,나 아무래도 혼자 갇힌 것 같애.,fear,sadness,2,fear,2,fear,2,fear,2,fear,2,48,female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19348,5fbe2db6cb503578af9eed5e,와 짜장면에서 벌레나왔어.,disgust,sadness,1,sadness,1,disgust,2,neutral,0,neutral,0,23,female
19363,5fbe306644697678c497c050,나 너무 우울해.,sadness,sadness,1,sadness,1,sadness,2,sadness,1,neutral,0,23,female
19369,5fbe313c44697678c497c05a,나 엘리베이터에 갇혔어.,fear,happiness,1,sadness,1,sadness,2,sadness,1,sadness,1,23,female
19372,5fbe2f8544697678c497c047,나 드디어 프로젝트 끝났어!,happiness,disgust,1,sadness,1,neutral,0,happiness,1,sadness,1,23,female


In [41]:
# drop duplicates
df = df.drop_duplicates(['발화문'], keep='first', ignore_index = True)
df.shape

(16137, 15)

In [42]:
# reset index
df.reset_index(drop=True)

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,5f4141e29dd513131eacee2f,헐! 나 이벤트에 당첨 됐어.,happiness,angry,2,surprise,2,happiness,2,happiness,2,happiness,2,48,female
1,5f4141f59dd513131eacee30,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,happiness,neutral,0,happiness,2,happiness,2,happiness,2,happiness,2,48,female
2,5f4142119dd513131eacee31,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,2,48,female
3,5f4142279dd513131eacee32,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,1,48,female
4,5f3c9ed98a3c1005aa97c4bd,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,neutral,happiness,2,happiness,1,happiness,2,happiness,1,happiness,1,48,female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16132,5fbe30cd4c55eb78bd7cee7c,좋은 생각이야. 괜찮은 운동 있어?,sadness,sadness,1,neutral,0,surprise,2,neutral,0,neutral,0,23,female
16133,5fbe30de44697678c497c054,어떤 홈 트레이닝이 있어?,sadness,neutral,0,fear,1,neutral,0,neutral,0,neutral,0,23,female
16134,5fbe25444c55eb78bd7cedb6,맞아. 나한테만 퉁명스럽고 일을 많이 시키더라고.,angry,sadness,1,sadness,1,neutral,0,sadness,1,neutral,0,23,female
16135,5fbe251044697678c497bfb8,하지만 기분이 나쁜 걸 어떡해?,angry,sadness,1,fear,1,sadness,2,sadness,1,neutral,0,23,female


### Choose the most dominant emotion

In [43]:
# check emotions list
df['1번 감정'].unique()

array(['angry', 'neutral', 'happiness', 'sadness', 'disgust', 'surprise',
       'fear'], dtype=object)

In [44]:
# select most common emotion among 5 emotions
from collections import Counter

emotion = []
for idx, row in df.iterrows():
    emotion_list = [row['1번 감정'], row['2번 감정'], row['3번 감정'], row['4번 감정'], row['5번 감정']]
    cnt = Counter(emotion_list)
    most = cnt.most_common(1)[0][0]
    emotion.append(str(most))

df['emotion'] = emotion
df[:5]

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별,emotion
0,5f4141e29dd513131eacee2f,헐! 나 이벤트에 당첨 됐어.,happiness,angry,2,surprise,2,happiness,2,happiness,2,happiness,2,48,female,happiness
1,5f4141f59dd513131eacee30,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,happiness,neutral,0,happiness,2,happiness,2,happiness,2,happiness,2,48,female,happiness
2,5f4142119dd513131eacee31,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,2,48,female,happiness
3,5f4142279dd513131eacee32,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,1,48,female,happiness
4,5f3c9ed98a3c1005aa97c4bd,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,neutral,happiness,2,happiness,1,happiness,2,happiness,1,happiness,1,48,female,happiness


In [45]:
# check each emotions by group
df['emotion'].value_counts()

neutral      5275
sadness      3033
happiness    2999
angry        1705
disgust      1323
fear          929
surprise      873
Name: emotion, dtype: int64

In [46]:
df[:5]

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별,emotion
0,5f4141e29dd513131eacee2f,헐! 나 이벤트에 당첨 됐어.,happiness,angry,2,surprise,2,happiness,2,happiness,2,happiness,2,48,female,happiness
1,5f4141f59dd513131eacee30,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,happiness,neutral,0,happiness,2,happiness,2,happiness,2,happiness,2,48,female,happiness
2,5f4142119dd513131eacee31,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,2,48,female,happiness
3,5f4142279dd513131eacee32,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,1,48,female,happiness
4,5f3c9ed98a3c1005aa97c4bd,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,neutral,happiness,2,happiness,1,happiness,2,happiness,1,happiness,1,48,female,happiness


### Map emotion to numerical labels

In [47]:
# only get needed data
df = df.loc[:, ["발화문", "emotion"]]

In [48]:
emotion_dict = {"neutral": 0,
               "sadness" : 1,
               "happiness" : 2,
               "angry" : 3,
               "disgust" : 4,
               "fear" : 5,
               "surprise" : 6}

In [49]:
import numpy as np
mapped_emotion = [emotion_dict.get(emotion, 0) for emotion in df["emotion"]]

In [50]:
df['emotion_num'] = mapped_emotion

In [51]:
# check emotion labeling
df['emotion_num'].tolist()[:20]

[2, 2, 2, 2, 2, 2, 1, 2, 0, 0, 1, 1, 1, 3, 3, 3, 2, 1, 1, 1]

In [52]:
# select columns needed for train
df = df.loc[:, ["발화문", "emotion_num"]]
df.columns = ["user_chat", "emotion_num"]

In [53]:
df

,user_chat,emotion_num
0,헐! 나 이벤트에 당첨 됐어.,2
1,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,2
2,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",2
3,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",2
4,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,2
...,...,...
16132,좋은 생각이야. 괜찮은 운동 있어?,0
16133,어떤 홈 트레이닝이 있어?,0
16134,맞아. 나한테만 퉁명스럽고 일을 많이 시키더라고.,1
16135,하지만 기분이 나쁜 걸 어떡해?,1


## 2-2. Add hiing chat data

In [54]:
df_chat = pd.read_csv("hiing_emotion_labeled_2to5.csv", encoding="cp949")
df_chat

,Column1,user_chat,emotion,degree
0,0,ㅎㅇㅎㅇ,Neutral,1
1,1,ㅎㅇㅎㅇ,Neutral,1
2,2,안녕~~~~,Neutral,1
3,3,너 누구야??,Neutral,1
4,4,싫은데,Anger,2
...,...,...,...,...
2385,2385,그레,Neutral,1
2386,2386,여자친구를 사귀고 싶은돼 모하궈,Neutral,1
2387,2387,혹시,Surprise,2
2388,2388,내 여자친구 될레,Neutral,1


In [55]:
# check values
df_chat.value_counts('emotion')

emotion
Neutral      1364
Happiness     450
Sadness       249
Anger         188
Surprise      137
Fear            2
dtype: int64

In [56]:
emotion_dict = {"Neutral": 0,
               "Sadness" : 1,
               "Happiness" : 2,
               "Anger" : 3,
               "Disgust" : 4,
               "Fear" : 5,
               "Surprise" : 6}

In [57]:
import numpy as np
mapped_emotion = [emotion_dict.get(emotion, 0) for emotion in df_chat["emotion"]]

In [58]:
df_chat['emotion_num'] = mapped_emotion

In [59]:
df_chat['emotion_num'].tolist()[:20]

[0, 0, 0, 0, 3, 0, 0, 3, 3, 2, 0, 0, 0, 0, 2, 2, 0, 3, 0, 2]

In [60]:
df_chat = df_chat.loc[:, ["user_chat", "emotion_num"]]
df_chat

,user_chat,emotion_num
0,ㅎㅇㅎㅇ,0
1,ㅎㅇㅎㅇ,0
2,안녕~~~~,0
3,너 누구야??,0
4,싫은데,3
...,...,...
2385,그레,0
2386,여자친구를 사귀고 싶은돼 모하궈,0
2387,혹시,6
2388,내 여자친구 될레,0


## 2-3. Split train & test data set

In [61]:
from sklearn.model_selection import train_test_split

In [62]:
# merge ai-hub data and hiing data
df = pd.concat([df, df_chat], axis = 0)
df = df.reset_index(drop=True)
df

,user_chat,emotion_num
0,헐! 나 이벤트에 당첨 됐어.,2
1,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,2
2,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",2
3,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",2
4,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,2
...,...,...
18522,그레,0
18523,여자친구를 사귀고 싶은돼 모하궈,0
18524,혹시,6
18525,내 여자친구 될레,0


In [63]:
df_train, df_test = train_test_split(df, test_size = 0.2, shuffle = True, random_state = 32)

In [64]:
print(len(df_train), len(df_test))

14821 3706


In [65]:
train_chat_list = df_train['user_chat'].tolist()
test_chat_list = df_test['user_chat'].tolist()
train_emotion_list = df_train['emotion_num'].tolist()
test_emotion_list = df_test['emotion_num'].tolist()

In [66]:
train_dataset = {"text" : train_chat_list,
      "label" : train_emotion_list}
test_dataset = {"text" : test_chat_list,
      "label" : test_emotion_list}

In [67]:
from datasets import DatasetDict, Dataset
train_dataset = Dataset.from_dict(train_dataset)
test_dataset = Dataset.from_dict(test_dataset)
dataset_dict = DatasetDict({"train": train_dataset, "test" : test_dataset})

In [68]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 14821
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3706
    })
})

## 2-4. Tokenize data

In [69]:
from transformers import BertForSequenceClassification
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels = 7 )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 14821
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3706
    })
})

In [71]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True, max_length=500)

tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/14821 [00:00<?, ? examples/s]

Map:   0%|          | 0/3706 [00:00<?, ? examples/s]

In [72]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 14821
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3706
    })
})


# 3. Train Model

In [73]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [74]:
# load metrics for evaluation
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [78]:
# huggingface login
from huggingface_hub import notebook_login
notebook_login()

In [79]:
from transformers import TrainingArguments, Trainer

repo_name = "emotion-bert-finetuning-18500"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=10,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset['train'],
   eval_dataset=tokenized_dataset['test'],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

Step,Training Loss
500,1.082200
1000,0.900300
1500,0.765300
2000,0.712300
2500,0.597800
3000,0.532000
3500,0.463400
4000,0.391900
4500,0.340800
5000,0.282000


In [ ]:
# upload model to huggingface
trainer.push_to_hub()

# 4. Evaluation

In [45]:
# download model from huggingface
from transformers import pipeline
model = pipeline(model='eunyounglee/emotion-bert-finetuning-3')

config.json:   0%|          | 0.00/995 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [58]:
# inference test
model(["너무 좋아~"])

[{'label': 'LABEL_1', 'score': 0.4120016098022461}]

In [ ]:
# get un-trained data
import pandas as pd
answer_df = pd.read_csv("./hiing_emotion_labeled_5.csv", encoding = "UTF-8")

In [ ]:
emotion_dict = {"Neutral": 0,
               "Sadness" : 1,
               "Happiness" : 2,
               "Anger" : 3,
               "Disgust" : 4,
               "Fear" : 5,
               "Surprise" : 6}

In [ ]:
# map emotion data to numbers
import numpy as np
mapped_emotion = [emotion_dict.get(emotion, 0) for emotion in answer_df["sentiment"]]
answer_df['emotion_num'] = mapped_emotion

In [46]:
# emotion labels
emotion_dict = {0 : "Neutral",
               1 : "Sadness",
               2 : "Happiness",
               3 : "Angry",
               4 : "Disgust",
               5 : "Fear",
               6 : "Surprise"}

In [ ]:
# predict emotion
predicted_list = []
for i in range(len(answer_df)):
    temp = model([answer_df.loc[i, "user_chat"]])[0]['label'][-1]
    predicted_list.append(temp)

In [ ]:
# calculate accuracy and f1 score
from datasets import load_metric

load_accuracy = load_metric("accuracy")
accuracy = load_accuracy.compute(predictions=predicted_list, references=answer_df['emotion_num'].tolist())["accuracy"]
f1 = load_f1.compute(predictions=predicted_list, references=answer_df['emotion_num'].tolist(), average='weighted')["f1"]
print("accuracy: ", accuracy, " f1: ", f1)